# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Question 1

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

"""

FOR THIS QUERY SESSIONID AND ITEMINSESSION ARE CONSIDERED AS THE PARTITION KEY. 
BECAUSE I NEED TO GET ARTIST NAME, SONG NAME OR SONG TITLE, LENGTH OF SONG WITH ITEM IN SESSION AND SESSION ID 
IN THE EVENT DATA TABLE FOR ANALYSIS FROM EQUALITY OPERATION.

WHERE:
ITEM IN SESSION : NO. OF ITEM PER TRANSACTION.
SESSION ID : TRANSACTION NO. PER USER

"""

query = "CREATE TABLE IF NOT EXISTS event_data_table "
query = query + """(sessionid int, iteminsession int, artist varchar, song_title varchar, length float, PRIMARY KEY(sessionid, iteminsession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

"""

FOR THIS QUERY  SESSION ID,  ITEM IN SESSION, ARTIST NAME, SONG NAME, SONG LENGTH ARE INSERTED INTO EVENT DATA TABLE
BECAUSE I NEED TO INSERT DATA PER ROW INTO EMPTY TABLE BEFORE.


"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into event_data_table (sessionid, iteminsession, artist, song_title, length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Question 1 Results

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
##Add in the SELECT statement to verify the data was entered into the table

"""
FOR THIS QUERY ARTIST NAME, SONG NAME AND SONG LENGTH ARE SELECTED WITH SESSION ID AND ITEM IN SESSION CONDITIONS
BECAUSE : I NEED TO GET CUSTOMER INFORMATIONS FROM SESSION ID AND ITEM IN SESSION CONDITION FOR EASY TO DATA ANALYSIS

IN THIS CASE: I GET CUSTOMER INFORMATIONS (EXAMPLE: ARTIST NAME, SONG NAME AND SONG LENGTH)
WHERE SESSION ID = 338 AND ITEM IN SESSION = 4 
(MEANING : USER OR CUSTOMER IN TRANSACTION 338 LISTEN FOURTH MUSIC IN LIST AS "Music Matters (Mark Knight Dub)")

"""

#Question 1
query = "SELECT artist, song_title, length from event_data_table where sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [13]:
"""
FOR CHECKING YOUR QUERY ARE CORRECT !

"""
file = 'event_datafile_new.csv'

test_data = pd.read_csv(file)
test_data.loc[(test_data['itemInSession'] == 4) & (test_data['sessionId'] == 338)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


#### Question 2

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

"""

FOR THIS QUERY USER ID AND SESSION ID ARE CONSIDERED AS THE PARTITION KEY AND ITEM IN SESSION IS CLUSTERING KEY 
BECAUSE I NEED TO GET ARTIST NAME, SONG NAME OR SONG TITLE AND USERNAME (FIRST AND LAST NAME) 
FOR EACH USER ID AND SESSION ID IN THE EVENT DATA TABLE FOR ANALYSIS 
FROM EQUALITY OPERATION OR FILTERING OPERATION (PARTITION KEY) AND SORTING OPERATION (CLUSTERING KEY).

WHERE:
USER ID : KEY OF CUSTOMER FOR USER IDENTIFIED PER TRANSACTION
SESSION ID : TRANSACTION NO. PER USER
ITEM IN SESSION : NO. OF ITEM PER TRANSACTION.
"""

query = "CREATE TABLE IF NOT EXISTS song_detail "
query = query + """(userid int, sessionid int, artist varchar, song_title varchar, firstname varchar, 
lastname varchar, iteminsession int, PRIMARY KEY((userid, sessionid), iteminsession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:

"""

FOR THIS QUERY  USER ID, SESSION ID, ARTIST NAME, SONG TITLE, FIRST NAME, LAST NAME, ITEM IN SESSION ARE INSERTED 
INTO SONG DETAIL TABLE
BECAUSE I NEED TO INSERT DATA PER ROW INTO EMPTY TABLE AND ORDER BY KEY BEFORE.

"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into song_detail (userid, sessionid, artist, song_title, firstname, lastname, iteminsession)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))
                    

#### Question 2 Results

In [16]:
"""

FOR THIS QUERY ARTIST NAME, SONG NAME, USERNAME (FIRST NAMD AND LAST NAME) ARE SELECTED 
WITH USER ID AND SESSION ID BY EQUALITY OPERATION AND ITEM IN SESSION BY SORTING OPERATION
BECAUSE : I NEED TO GET CUSTOMER INFORMATIONS FROM USER ID AND SESSION ID AND SORTING BY ITEM IN SESSION FOR DATA ANALYSIS

IN THIS CASE: I GET CUSTOMER INFORMATIONS (EXAMPLE: ARTIST NAME, SONG NAME AND USERNAME)
WHERE USER ID = 10 AND SESSION ID = 182 AND SORTING BY ITEM IN SESSION
(MEANING : NO. OF USER ID 10 IN TRANSACTION 182 THAT LISTENING MUSICS IN LIST ORDER BY ITEM IN SESSION (START TO END OR END TO START))

"""

#Question 2

query = "SELECT artist, song_title, firstname, lastname from song_detail where userid = 10 and sessionid = 182 ORDER BY iteminsession ASC"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [17]:
"""
FOR CHECKING YOUR QUERY ARE CORRECT !

"""

test_data[(test_data['sessionId'] == 182) & (test_data['userId'] == 10)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


#### Question 3

In [34]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

"""
FOR THIS QUERY SONG TITLE  IS CONSIDERED AS THE PARTITION KEY , USER ID AND USERNAME ARE CLUSTERING KEY 
BECAUSE I NEED TO GET USERNAME (FIRST AND LAST NAME) THAT LISTENING MUSIC (SONG TITLE)
WITH SONG TITLE AND USER ID IN THE EVENT DATA TABLE FOR DATA ANALYSIS AND UPDATING DATA SUPPORT IN SAME CUSTOMERS .

IN THIS CASE, SONG TITLE HAS TO USE EQUALITY OPERATION OR FILTERING OPERATION (PARTITION KEY).

WHERE:
SONG TITLE : SONG NAME
USER ID : KEY OF CUSTOMER FOR USER IDENTIFIED PER TRANSACTION
USERNAME : FIRST NAME + LAST NAME

AND

FOR THIS QUERY  SONG TITLE, USER ID, ARTIST NAME, FIRST NAME, LAST NAME, SESSION ID, ITEM IN SESSION ARE INSERTED 
INTO USER SONG LIST TABLE
BECAUSE I NEED TO INSERT DATA PER ROW INTO EMPTY TABLE AND ORDER BY KEY BEFORE.

PROS : THE SAME NAME AND DIFFERENT CUSTOMER CASE ARE INCLUDED BY USER ID KEY

"""

query = "CREATE TABLE IF NOT EXISTS user_song_list "
query = query + """(song_title varchar, userid int, artist varchar, firstname varchar, lastname varchar, 
sessionid int, iteminsession int, PRIMARY KEY((song_title), userid, firstname, lastname))"""

try:
    session.execute(query)
except Exception as e:
    print(e)


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into user_song_list (song_title, userid, artist, firstname, lastname, sessionid, iteminsession)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[4], int(line[8]), int(line[3])))

#### Question 3 Results

In [38]:
"""

FOR THIS QUERY USERNAME (FIRST NAMD AND LAST NAME) ARE SELECTED 
WITH SONG TITLE BY EQUALITY OPERATION
BECAUSE : I NEED TO GET CUSTOMER INFORMATIONS FROM SONG TITLE FOR DATA ANALYSIS AND CUSTOMER
WHO WERE LISTENED THIS SONG

IN THIS CASE: I GET CUSTOMER INFORMATIONS (EXAMPLE: USERNAME)
WHERE SONG TITLE = 'All Hands Against His Own'
(MEANING : CUSTOMER WHO WERE LISTENED MUSIC 'All Hands Against His Own')

PROS : THE SAME NAME AND DIFFERENT CUSTOMER CASE ARE INCLUDED BY USER ID KEY

"""

#Question 3

query = "SELECT firstname, lastname from user_song_list where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [21]:
"""
FOR CHECKING YOUR QUERY ARE CORRECT !

"""

test_data[test_data['song'] == "All Hands Against His Own"]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [39]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table event_data_table "

try:
    session.execute(query)
except Exception as e:
    print(e)


In [40]:
query = "drop table song_detail "

try:
    session.execute(query)
except Exception as e:
    print(e)

In [41]:
query = "drop table user_song_list "

try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()